## LM-Eval-Harness 사용하기

In [ ]:
lm_eval --model vllm \
    --model_args pretrained=kakaocorp/kanana-1.5-2.1b-instruct-2505,dtype=auto \
    --tasks kmmlu_direct \
    --log_samples \
    --output_path results \
    --batch_size auto

# |            Groups             |Version|Filter|n-shot|  Metric   |   |Value |   |Stderr|
# |-------------------------------|------:|------|------|-----------|---|-----:|---|-----:|
# |kmmlu_direct                   |      2|none  |      |exact_match|↑  |0.1355|±  |0.0018|
# | - kmmlu_direct_applied_science|      2|none  |      |exact_match|↑  |0.1555|±  |0.0034|
# | - kmmlu_direct_humss          |      2|none  |      |exact_match|↑  |0.0895|±  |0.0039|
# | - kmmlu_direct_other          |      2|none  |      |exact_match|↑  |0.1168|±  |0.0035|
# | - kmmlu_direct_stem           |      2|none  |      |exact_match|↑  |0.1519|±  |0.0036|

# |            Groups             |Version|Filter|n-shot|  Metric   |   |Value |   |Stderr|
# |-------------------------------|------:|------|------|-----------|---|-----:|---|-----:|
# |kmmlu_direct                   |      2|none  |      |exact_match|↑  |0.3837|±  |0.0026|
# | - kmmlu_direct_applied_science|      2|none  |      |exact_match|↑  |0.3544|±  |0.0044|
# | - kmmlu_direct_humss          |      2|none  |      |exact_match|↑  |0.4316|±  |0.0069|
# | - kmmlu_direct_other          |      2|none  |      |exact_match|↑  |0.3844|±  |0.0052|
# | - kmmlu_direct_stem           |      2|none  |      |exact_match|↑  |0.3925|±  |0.0048|

In [ ]:
lm_eval --model vllm \
    --model_args pretrained=amphora/fc-proj1-test01,dtype=auto \
    --tasks kmmlu_direct \
    --log_samples \
    --output_path results \
    --batch_size auto

# |            Groups             |Version|Filter|n-shot|  Metric   |   |Value |   |Stderr|
# |-------------------------------|------:|------|------|-----------|---|-----:|---|-----:|
# |kmmlu_direct                   |      2|none  |      |exact_match|↑  |0.4009|±  |0.0026|
# | - kmmlu_direct_applied_science|      2|none  |      |exact_match|↑  |0.3834|±  |0.0045|
# | - kmmlu_direct_humss          |      2|none  |      |exact_match|↑  |0.3940|±  |0.0068|
# | - kmmlu_direct_other          |      2|none  |      |exact_match|↑  |0.4121|±  |0.0052|
# | - kmmlu_direct_stem           |      2|none  |      |exact_match|↑  |0.4153|±  |0.0048|

In [ ]:
## 자체 코드 사용하기

In [ ]:
python evaluation.py \
  --model amphora/fc-proj1-test01 \
  --dataset KMMLU_Redux \
  --temperature 0.7 \
  --top_p 0.9 \
  --reasoning False \
  --max_tokens 8192

python evaluation.py \
  --model amphora/fc-proj1-test01 \
  --dataset MCLM \
  --temperature 0.7 \
  --top_p 0.9 \
  --reasoning False \
  --max_tokens 8291

python evaluation.py \
  --model amphora/fc-proj1-test01 \
  --dataset HRB1_0 \
  --temperature 0.7 \
  --top_p 0.9 \
  --reasoning False \
  --max_tokens 8192

python evaluation.py \
  --model pretrained=kakaocorp/kanana-1.5-2.1b-instruct-2505 \
  --dataset KMMLU_Redux \
  --temperature 0.7 \
  --top_p 0.9 \
  --reasoning False \
  --max_tokens 8192

python evaluation.py \
  --model pretrained=kakaocorp/kanana-1.5-2.1b-instruct-2505 \
  --dataset MCLM \
  --temperature 0.7 \
  --top_p 0.9 \
  --reasoning False \
  --max_tokens 8291

python evaluation.py \
  --model pretrained=kakaocorp/kanana-1.5-2.1b-instruct-2505 \
  --dataset HRB1_0 \
  --temperature 0.7 \
  --top_p 0.9 \
  --reasoning False \
  --max_tokens 8192

In [23]:
# KMMLU-Redux Evaluation
import pandas as pd 
from math_verify import parse

df = pd.read_csv('HRB1_0-amphora_fc-proj1-test01.csv')

output = []
for _,row in df.iterrows():
    try:
        response = row.response
        resp = parse(response)
        output.append(resp[-1])
    except:
        output.append('-1')
df['pred'] = output     


# 1) Define mappings
num2letter = {1: "A", 2: "B", 3: "C", 4: "D", 5:'E'}
letter2num = {v: k for k, v in num2letter.items()}

# 2) Normalize both sides to numbers (1–4) in a new column pred_num
def to_pred_num(x):
    # if it’s already an integer or a numeric string
    if isinstance(x, (int, float)) or (isinstance(x, str) and x.isdigit()):
        try:
            return int(x)
        except ValueError:
            pass
    # if it’s one of the letters A–D
    if isinstance(x, str) and x.upper() in letter2num:
        return letter2num[x.upper()]
    # otherwise
    return None

df["pred_num"] = df["pred"].apply(to_pred_num)

# 3) Optionally also create gold_letter if you want letter view
df["gold_letter"] = df["gold"].map(num2letter)

# 4) Now mark correctness
df["correct"] = df["pred"] == df["gold"].astype(str) 

# 5) Overall accuracy
overall_acc = df["correct"].mean()
print(f"Overall accuracy: {overall_acc:.2%}")

# 6) Accuracy by category
acc_by_cat = df.groupby("category")["correct"].mean().sort_values(ascending=False)
print("\nAccuracy by category:")
print(acc_by_cat)

Overall accuracy: 62.29%

Accuracy by category:
category
HI     0.840426
rw     0.765432
sn     0.758170
lw     0.627219
RC     0.474273
KGK    0.318182
Name: correct, dtype: float64


In [24]:
# KMMLU-Redux Evaluation
import pandas as pd 
from math_verify import parse

df = pd.read_csv('HRB1_0-kakaocorp_kanana-1.5-2.1b-instruct-2505.csv')

output = []
for _,row in df.iterrows():
    try:
        response = row.response
        resp = parse(response)
        output.append(resp[-1])
    except:
        output.append('-1')
df['pred'] = output     


# 1) Define mappings
num2letter = {1: "A", 2: "B", 3: "C", 4: "D", 5:'E'}
letter2num = {v: k for k, v in num2letter.items()}

# 2) Normalize both sides to numbers (1–4) in a new column pred_num
def to_pred_num(x):
    # if it’s already an integer or a numeric string
    if isinstance(x, (int, float)) or (isinstance(x, str) and x.isdigit()):
        try:
            return int(x)
        except ValueError:
            pass
    # if it’s one of the letters A–D
    if isinstance(x, str) and x.upper() in letter2num:
        return letter2num[x.upper()]
    # otherwise
    return None

df["pred_num"] = df["pred"].apply(to_pred_num)

# 3) Optionally also create gold_letter if you want letter view
df["gold_letter"] = df["gold"].map(num2letter)

# 4) Now mark correctness
df["correct"] = df["pred"] == df["gold"].astype(str) 

# 5) Overall accuracy
overall_acc = df["correct"].mean()
print(f"Overall accuracy: {overall_acc:.2%}")

# 6) Accuracy by category
acc_by_cat = df.groupby("category")["correct"].mean().sort_values(ascending=False)
print("\nAccuracy by category:")
print(acc_by_cat)

Overall accuracy: 49.02%

Accuracy by category:
category
HI     0.718085
sn     0.633987
rw     0.582716
lw     0.562130
RC     0.315436
KGK    0.284091
Name: correct, dtype: float64


In [16]:
# KMMLU-Redux Evaluation
import pandas as pd 
from math_verify import parse

df = pd.read_csv('KMMLU_Redux-amphora_fc-proj1-test01.csv')

output = []
for _,row in df.iterrows():
    try:
        response = row.response
        resp = parse(response)
        output.append(resp[-1])
    except:
        output.append('-1')
df['pred'] = output     


# 1) Define mappings
num2letter = {1: "A", 2: "B", 3: "C", 4: "D", 5:'E'}
letter2num = {v: k for k, v in num2letter.items()}

# 2) Normalize both sides to numbers (1–4) in a new column pred_num
def to_pred_num(x):
    # if it’s already an integer or a numeric string
    if isinstance(x, (int, float)) or (isinstance(x, str) and x.isdigit()):
        try:
            return int(x)
        except ValueError:
            pass
    # if it’s one of the letters A–D
    if isinstance(x, str) and x.upper() in letter2num:
        return letter2num[x.upper()]
    # otherwise
    return None

df["pred_num"] = df["pred"].apply(to_pred_num)

# 3) Optionally also create gold_letter if you want letter view
df["gold_letter"] = df["gold"].map(num2letter)

# 4) Now mark correctness
df["correct"] = df["pred"] == df["gold"].astype(str) 

# 5) Overall accuracy
overall_acc = df["correct"].mean()
print(f"Overall accuracy: {overall_acc:.2%}")

# 6) Accuracy by category
acc_by_cat = df.groupby("category")["correct"].mean().sort_values(ascending=False)
print("\nAccuracy by category:")
print(acc_by_cat)

Overall accuracy: 47.16%

Accuracy by category:
category
전기.전자     0.568182
정보통신      0.526786
환경.에너지    0.481557
화학        0.465608
건설        0.459082
안전관리      0.445151
기계        0.422053
재료        0.408163
식품.가공     0.388889
Name: correct, dtype: float64


## 정성평가 진행하기

In [ ]:
vllm serve amphora/fc-proj1-test01 \
  --dtype auto \
  --api-key token-abc123

In [40]:
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="token-abc123",
)

In [37]:
prompts = [
    "'돛단배'로 삼행시 해줘.",
    "<h1>Ⅲ. 실험 및 결과 고찰</h1> <h2>3-1 제작 및 측정</h2> <p>단락핀에 슬릿을 부가하여 설계한 안테나의 복사판은 밀링머신을 사용하여 \\( 0.2 \\mathrm { ~mm } \\) 두께롤 갖는 동판 위에 슬릿을 깍고, 유전체 기판은 광에칭 방법으로 급전 선로와 접지면을 제작하였으며, 유전체 기판 위에 동판을 정확한 위치에 배열하여 단락핀과 접지면올 연결하고, 급전선로의 종단부분과 복사판을 연결함으로써 제작하였다. 단락핀의 지름은 \\( 0.5 \\mathrm { ~mm } \\)이고, 단락핀에 연견된 접지면의 크기는 \\( 1.5 \\mathrm { ~mm } \\times 12 \\mathrm { mm } (W \\times L) \\) 이며, 슬릿의 폭은 \\( 0.5 \\mathrm { ~mm } \\) 이다. 그림 6은 제작된 안테나의 실체 모습이다.</p> <p>제작된 안테나의 반사손실, 입력 임피던스를 HP-8753ES vector network analyzer로 측정하여 그 결과를 그림 7과 그림 8에 나타내었다.</p> <p>측정 결과, \\( 1.934 \\mathrm { GHz } \\)에서 반사 손실이 \\( -15.375 \\mathrm { ~dB } \\)이고, 안테나의 입력 임피던스는 \\(40.816 \\ -j12.637 \\Omega \\)이며, VSWR이 2.5이하에서 대역폭이 \\( 803 \\mathrm { MHz } (40.8 \\%) \\) 로 PCS/IMT-2000 및 Wibro 대역을 만족하였다.</p> <p>안테나의 복사 패턴은 satimo stargate 32 장비로 E 및 H 패턴을 측정하였다. 그림 9는 \\( 1,800 \\mathrm { MHz } , 2010 \\mathrm { MHz } , 2,300 \\mathrm { MHz } \\)에서의 안테나의 복사 패턴이고, 이득은 PCS 대역에서 \\( 1.52 \\mathrm { dBi } , lMT-2000 대역에서는 \\( 2.62 \\mathrm { dBi } \\) 그리그 Wibro 대여에서는 \\( 1.55 \\mathrm { dBi } \\)의 최대값을 얻었다.</p> <h2>3-2 결과고찰</h2> <p>제작된 안테나의 주파수 특성은 시뮬레이션 결과와 비교적 일치했으나, 시뮬레이선보다 \\( 36 \\mathrm { MHz } \\) 더 넓은 대역폭을 가졌다. 이것은 안테나 제작시 급전 선로와 복사판 간의 연결 및 접지면과 단락핀에 연결에 의한 제작 및 측정상의 오차뿐만 아니라 시뮬레이션 오차로도 생각된다. 시뮬레이션 결과와 측정 결과를 표 1에 비교하였다.</p> <h1>Ⅳ. 결 론</h1> <p>본 논문에서는 크기가 작고, 제작이 용이하며, 구조가 간단하며, PCS/IMT-2000 및 Wibro 대역을 포함 하는 광대역 내장형 안테나를 설계하고 제작하였다. 설계된 안테나의 크기를 줄이기 위해 단락핀을 사용했으며, 광대역 특성을 얻기 위해 슬릿을 부가하였다. 단락핀의 위치에 따라 안테나의 동작 주파수를 결정할 수 있으며, 단락핀과 슬릿 간의 간겨 및 슬릿의 길이에 따라 안테나의 대역폭을 결정할 수 있었다. 측정결과 \\( 1.934 \\mathrm { GHz } \\)에서 대역폭은 \\( -7.5 \\mathrm { ~dB } \\) 기준으로 \\( 803 \\mathrm { MHz } \\)의 광대역 특성을 얻었으며, 이득은 PCS 대역에서 \\( 1.52 \\mathrm { dBi } \\), IMT-2000 대역에서는 \\( 2.62 \\mathrm { dBi } \\) 그리고 Wibro 대역에서는 \\( 1.55 \\mathrm { dBi } \\) 의 최대값을 얻었다. 제안된 안테나의 크기는 \\( 30 \\mathrm { ~mm } \\times 10 \\mathrm { ~mm } \\times \\ 0.2 \\mathrm { ~mm } \\)로 이동통신 단말기 내장형 안테나로 적합함을 알 수 있었다.</p>\n\n<h1>Ⅰ. 서 론</h1> <p>무선통신 기술의 급격한 발전으로 이동통신 단말기에 적합한 새로운 안테나 개발이 필수적이며 특히, 내장형 안테나에 대한 연구가 활발히 진행되고 있다.</p> <p>내장형 안테나는 외장형 안테나와 비교해서 쉽게 깨지거나 손상되지 않으며, 단말기의 소형화가 가능하며, 기존의 외장형 모노폴 안테나가 가지고 있던 무지향성 방사 패턴이 인체 두부에 미치는 영향을 고려할 때, 내장형 안테나는 전자표 흡수율(SAR: Specific Absorption Rate)을 크게 개선할 수 있는 장점이 있다. 그러나 이동통신 단말기에 내장하기 위한 안테나의 크기는 제한되어 있기 때문에, 안테나의 성능을 높이고, 이중 대역 또는 다중 대역을 설계시 두 개의 주파수에 대한 공진을 얻는다 할지라도 각각의 통신 대역에서 요구되는 대여폭을 만족시키는 것은 어려울 뿐만 아니라 광대역 특성을 얻는 것도 어렵다.</p> <p>이동통신 단말기 내장형 안테나에 적합한 형태로 세라믹 칩 안테나, 평면 역 F형 안테나 및 평면형 모노폴 안테나 등이 있다. 유전율이 높은 세라믹을 이용한 칩 안테나는 소형화의 장점이 있지만, 충격에 약하고, 제작비용이 높고, 제작이 완료된 이후에 칩상의 패턴을 조절하는 것이 쉽지 않다. 또한 높은 유전율에 의한 협대역 특성을 갖는 단점이 있다. 평면역 F형 안테나는 소형, 경량이면서, 제작이 쉽고, 안테나 특성이 우수하나 안테나의 높이가 낮아질수록 커패시턴스의 성분이 감소해 대역폭이 좁아지는 특성이 있기 때문에, 안테나의 높이를 줄이는 경우, 협소한 대역폭을 갖는 단점이 있다. 광대역 또는 이중 대역에서 동작하는 평면형 모노폴 안테나는 전체적으로 안테나의 크기가 큰 단점이 있다. 따라서 안테나의 크기를 줄이기 위해 단락핀을 사용하는 방법, 매우 높은 유전율을 갖는 기판을 사용하는 방법, 적층하는 방법 그리고 구조를 변형시키는 방법 등이 사용되고 있다.</p> <p>본 논문에서는 소형, 경량이면서 PCS/IMT-2000 및 Wibro 대역에서 동작하는 광대역 특성을 갖는 새로운 내장형 안테나를 설계하였다. 안테나의 크기를 줄이기 위해 단락핀을 사용하였으며, 광대역 특성을 얻기 위해 슬릿을 부가하였다. 복사판 내에서 단락핀의 위치, 단락핀과 슬릿간의 간격 및 슬릿의 길이에 대한 안테나의 특성을 유한 적분법(finite integration algorithm)과 구조 근사법(perfect boundary approximation)을 기본 이론으로 하는 CST Micro-wave Studio 상용 프로그램을 이용하여 해석하였다. 또한, 설계된 안테나를 제작하여 안테나의 반사 손실, 입력 임피던스, 복사 패턴 및 이득을 측정하였다.</p> <h1>Ⅱ. 소형 광대역 내장형 안테나의 설계</h1> <p>그림 1은 내장형 안테나 설계를 위한 기본 구조로서 안테나의 복사판은 \\( 0.2 \\mathrm { ~mm } \\) 두께를 갖는 동판으로 가로, 세로 길이는 \\( 30 \\mathrm { ~mm } \\times 10 \\mathrm { ~mm } \\) 로 고정하였으며, 유전율이 4.7, 기판의 두께가 \\( 0.7874 \\mathrm { ~mm } \\)인 FR4 기판 위에 복사판을 올려놓았다. 안테나의 임피던스 대역폭을 증가시키기 위해 복사판 아래의 유전체 기판의 접지면을 제거하였으며, 복사판과 접지면 사이의 간격은 \\( 2 \\mathrm { ~mm } \\) 로 하였으며, \\( 50 \\Omega \\) 마이크로스트립 급전 선로에 의해 직접 급전하였다. 이때 급전 선로의 길이는 \\( 68 \\mathrm { ~mm } \\)로 하였다.</p>\n\n<h2>2-1 단락핀에 의한 안테나의 특성</h2> <p>안테나의 크기를 줄이는 방법에는 유전율이 높은 기판을 사용하는 방법, 단락핀을 이용하는 방법, 안테나의 패턴 모양을 최적화하여 전기적 길이를 증가시키는 방법, 그리고 이들 방법을 조합하는 등 여러 가지 방법이 있다.</p> <p>본 논문에서는 안테나의 크기를 줄이기 위해 단락핀을 사용하였으며, 이때 단락핀은 안테나 내에서 인덕턴스 성분으로 동작하여 안테나의 공진 주파수를 낮아지게 한다. 단락핀의 위치 변화에 따른 안테나의 특성을 CST Microwave Studio 상용 프로그램을 이용하여 계산하였다.</p> <p>단락핀의 위치 \\( S_ { p } (x, y) \\) 를 (1,0.5)에서 \\(+ x\\) 방향으로 \\( 2 \\mathrm { ~mm } , +y\\) 방향으로 \\( 1 \\mathrm { ~mm } \\)씩 각각 변화시켰을 때의 동작 주파수를 계산하였으며, 그 결과 단락핀의 위치가 \\( + x \\) 방향으로 이동할수록 동작 주파수가 올라가고, \\( y \\) 방향으로 이동할수록 동작 주파수가 낮아져 단락핀의 위치가 (1,9.5)에서 최저 주파수 \\( 1.486 \\mathrm { GHz } \\)를 얻었다. 또한 단락핀의 위치 \\(y\\)가 5.5 이상일 때 \\(-10 \\mathrm { dB } \\) 이하의 반사손실을 얻었으며, \\(x\\)가 5에서 13 사이에 놓여졌을 때 설계하고자 하는 주파수에서 공진이 일어난다. 그러나 대부분의 경우 대역폭이 \\( -10 \\mathrm { ~dB } \\)를 기준으로 \\( 10 \\% \\) 이하를 얻었으며, 이는 단일대역이 아닌 다중대역에서 동작하기 위해 요구되는 대역폭을 만족시키기는 어럽다.</p> <p>그림 2(a)는 단락핀의 위치가 \\(+x\\) 방향으로 이동할 때의 동작 주파수의 변화이고, 그림 2(b)는 \\(+y\\) 방향으로 이동할 때의 동작 주파수의 변화를 나타내었다.</p> <h2>2-2 단락핀과 슬릿의 결합에 의한 안테나의 특성</h2> <p>다중대역에서 동작하기 위해 넓은 대역폭이 요구되는데, 본 논문에서는 광대역 특성을 얻기위해 단락핀 아래에 직사각형 슬릿을 부가하여, 단락핀을 고정시켜 놓고 단락핀과 슬릿간의 간격 및 슬릿의 길이에 대한 안테나의 특성을 계산하였다.</p> <p>그림 3과 같이 \\( 1.759 \\mathrm { GHz } \\)에서 공진이 일어나는 위치인 (5,5.5)에 단락핀을 고정시켜 놓고, 단락핀과 슬릿의 간격 \\( \\mathrm { d } \\)는 단락핀 아래로 \\( 0.5 \\mathrm { ~mm } \\) 위치에서 \\(1 \\mathrm { mm } \\) 간격으로 변화시켰으며, 슬릿의 길이는 \\( 1 \\mathrm { ~mm } \\)부터 \\( 29 \\mathrm { ~mm } \\)까지 변화시켰다. 그림 4는 단락핀과 슬릿 간의 간격에 따른 동작 주파수의 변화를 나타내었으며, 그림 5는 단락핀과 슬릿의 길이에 따른 시뮬레이션 결과를 나타내었다.</p>\n\n<p>그 결과, 단락핀과 슬릿의 길이 \\( L_ { S } \\)를 고정시켜 놓고, 단락핀과 슬릿의 간격 \\( \\mathrm { d } \\)를 \\( 0.5 \\mathrm { ~mm } , 1.5 \\mathrm { ~mm } , 2.5 \\mathrm { ~mm } , 3.5 \\mathrm { ~mm } , 4.5 \\mathrm { ~mm } \\)로 변화시켰을 때, 동작 주파수의 변화는 거의 없었다. 그러나 대역폭의 경우, 단락핀과 슬릿간의 간격이 멀어질수록 대역폭이 줄어들었다. 이는 단락핀과 슬릿간의 간격에 따른 커패시턴스 성분의 변화로 인해 대역폭이 변화됨을 알 수 있다.</p> <p>또한 단락핀과 슬릿간의 간격 \\( \\mathrm { d } \\)는 고정시켜 놓고, 슬릿의 길이 \\( L_ { s } \\)를 \\( 1 \\mathrm { ~mm } \\)부터 \\( 29 \\mathrm { ~mm } \\)까지 변화시켰을 때, 동작 주파수의 변화는 슬릿의 길이가 \\( 15 \\mathrm { ~mm } \\)까지는 거의 일정하다가 \\( 15 \\mathrm { ~mm } \\) 이상일 때 동작 주파수는 점점 높아지는 경향이 있다. 그림 5에서 \\( \\mathrm { d } \\) 가 \\( 1.5 \\mathrm { ~mm } \\)인 경우에 \\( L_ { s } \\)가 \\( 3 \\mathrm { ~mm } \\)일 때 가장 낮은 주파수 \\( 1.731 \\mathrm { GHz } \\)에서 \\( L_ { s } \\)가 \\( 29 \\mathrm { ~mm } \\)일 때 가장 높은 주파수 \\( 1.857 \\mathrm { GHz } \\)로 주파수가 \\( 126 \\mathrm { MHz } \\) 상향됨을 알 수 있었다. 대역폭도 슬릿의 길이가 길어질수록 증가하는데 이는 슬릿의 길이에 의해 형성되는 추가 공진율 결합함으로써 더 넓은 대역폭을 얻을 수 있었다.</p> <p>본 논문에서는 위의 시뮬레이션 결과로부터 단락핀의 위치 \\( S_ { p } ( \\mathrm { x } , \\mathrm { y } ) \\)는 \\( (5,5.5) \\) 이고, 슬릿의 길이 \\( L_ { S } \\)는 \\( 29 \\mathrm { ~mm } \\), 단락핀과 슬릿간의 간격 \\( \\mathrm { d } \\) 는 \\( 1.5 \\mathrm { ~mm } \\)로 하여 안테나를 설계하였으며, 시뮬레이션 결과, \\( 1.859 \\mathrm { GHz } \\)에서 반사 손실이 \\( -15.19 \\mathrm { ~dB } \\) 이며, 대역폭(VSWR< \\( 2.5) \\)은 \\( 767 \\mathrm { MHz } (38.5 \\%) \\)를 얻었다.</p>\n\n전문 분야 대학원생을 위해 300단어 분량의 요약을 작성하세요. 섹션 4개: '배경', '문헌적 공백', '접근 방법', '핵심 시사점'. 어조: 정확하고 기술적이며 기본 도메인 지식을 가정하세요.",
    "언제 취직할 수 있을까?",
    '주어진 프롬프트에서 생성된 실행 가능한 Python 함수를 생성하세요. 함수는 표준 입력을 입력으로 받아 출력을 출력해야 합니다. 정의 후 함수를 호출하세요. 당신은 물볼링 훈련에 있습니다. 왼손으로 플레이하는 사람은 l명, 오른손으로 플레이하는 사람은 r명, 왼손과 오른손 모두로 플레이할 수 있는 사람은 a명입니다.\n\n코치는 짝수의 선수로 구성된 팀을 만들기로 결정했습니다. 정확히 절반의 선수는 오른손으로, 정확히 절반의 선수는 왼손으로 플레이해야 합니다. 한 선수는 자신의 손 중 하나만 사용해야 합니다.\n\n양손잡이는 오른손과 왼손 모두로 잘 플레이합니다. 팀에서 양손잡이는 왼손이나 오른손으로 플레이할 수 있습니다.\n\n왼손과 오른손을 사용하는 선수의 수가 각각 같은 팀의 최대 가능한 크기를 찾아주세요.\n\n\n-----입력-----\n\n유일한 줄에는 세 개의 정수 l, r, a (0 ≤ l, r, a ≤ 100)가 포함되어 있습니다. 이는 훈련 중인 왼손잡이의 수, 오른손잡이의 수, 양손잡이의 수입니다.\n\n\n-----출력-----\n\n단일 짝수 정수를 출력하세요 — 팀의 최대 선수 수입니다. 팀에 0명의 선수만 있을 수도 있습니다.\n\n\n-----예제-----\n입력\n1 4 2\n\n출력\n6\n\n입력\n5 5 5\n\n출력\n14\n\n입력\n0 2 0\n\n출력\n0\n\n\n\n-----노트-----\n\n첫 번째 예제에서는 6명의 선수로 구성된 팀을 만들 수 있습니다. 왼손잡이 한 명과 두 명의 양손잡이를 왼손으로 플레이하게 하고, 세 명의 오른손잡이를 오른손으로 플레이하게 해야 합니다. 남은 한 사람은 팀에 포함될 수 없습니다.\n\n두 번째 예제에서는 14명의 사람으로 구성된 팀을 만들 수 있습니다. 다섯 명의 왼손잡이, 다섯 명의 오른손잡이, 두 명의 양손잡이를 왼손으로 플레이하게 하고 두 명의 양손잡이를 오른손으로 플레이하게 해야 합니다.',
    "전문 분야 대학원생을 위해 300단어 분량의 요약을 작성하세요. 섹션 4개: '배경', '문헌적 공백', '접근 방법', '핵심 시사점'. 어조: 정확하고 기술적이며 기본 도메인 지식을 가정하세요.\n\n<h1>1. 서 론</h1> <p>영구자석 동기전동기는 넓은 운전 영역에 걸쳐서 높은 효율을 유지하는 특성으로 인하여 산업계에서 고효율이 요구되는 응용 분야에 최적의 전동기로 각광받고있다. 특히, 집중권 고정자와 표면부착형 페라이트 영구자석 회전자로 구성된 영구자석 동기전동기는 비교적 저가로 양산이 가능하기 때문에 팬(Fan) 구동 등 의 응용 분야에 널리 사용되어지고 있다.</p> <p>표면부착형 영구자석 동기전동기를 단위 전류 당 최대 토크 운전하기 위해서는 d축(Direct-axis) 전류를 영(零)으로 제어해야 한다. 전류 제어를 위해서는 전류 검출 센서 및 고속 연산이 가능한 마이크로프로세서가 필수적으로 요구되며, 이는 전동기 구동 시스템의 구현 비용을 상승시켜 영구자석 동기전동기의 산업계 확산에 장애 요소로 작용하고 있다.</p> <p>산업계에서는 전류 제어에 따른 비용 상승 문제를 해결하기 위해 전류 제어 없이 전압 제어만을 수행하는 경우를 쉽게 찾아 볼 수 있다. 이러한 경우, 단위 전류 당 최대 토크 운전을 위한 전압 지령의 위상각을 룩업(Look-up) 테이블 형태로 만들어서 표면부착형 영구자석 동기전동기에 인가되는 전압의 위상을 제어 하는 것이 일반적이다. 그러나 이 방식은 다양한 운전 조건에 대한 전압 지령의 위상각을 테이블로 만드는 것이 현실적으로 어렵다는 문제점을 지니고 있다.</p> <p>또 다른 방법으로, 전동기 구동을 위한 인버터 회로에 사용되는 션트(Shunt) 저항으로부터 전동기에 공급되는 상전류의 피크(peak)값 발생 시점을 검출하고, 이 시점과 영구자석 동기전동기의 회전자 위치 검출을 위한 홀센서의 출력이 변화되는 시점이 일치되도록 전압 지령의 위상각을 제어하는 방법이 있다. 이 방식에서는 션트 저항으로부터 전류의 피크값 발생 시점을 검출하는데 있어서 인버터의 스위칭으로 인한 전기적 노이즈의 영향을 받기가 쉽기 때문에 노이즈 필터링(Filtering)에 많은 노력이 필요하다.</p> <p>전류 센서 없이 전동기의 수학적 모델로부터 추정된 전류를 기반으로 전류 제어를 하는 방법도 제안되었다. 그러나 이 방식은 추정 전류에 오차가 포함될 경우 데드타임(Dead-time) 보상이 잘못되어 오히려 제어성능이 더 나빠지고, 스위칭 주기마다 전류 추정 및 전류 제어 알고리즘을 수행해야 하므로 고속 연산이 가능한 마이크로프로세서를 필요로 하는 단점을 가진다.</p> <p>본 논문에서는 전류 제어나 전류 검출 없이 단순히 전압 지령의 위상각을 제어하여 표면부착형 영구자석 동기전동기의 단위전류 당 최대토크 운전을 달성할 수 있는 새로운 제어 방법을 제안하고자 한다. 제안된 방법은 영구자석 동기전동기의 수학적 모델로부터 d축 전류를 추정하고, 추정된 d축 전류가 영(零)이 되도록 적분 제어를 통해 전압 지령의 위상각을 조절해 간다. 또한 본 논문은 데드타임에 의한 전류 추정 오차를 보상할 수 있는 방법에 대해서도 제안하고자 한다. 실험을 통해 제안된 방법의 과도 상태 및 정상 상태에서의 특성을 평가한다.</p>\n\n<h1>3. 데드타임에 의한 전류 추정 오차의 보상</h1> <p>고성능이 요구되는 서보 분야에서는 데드타임으로 인해 발생하는 전압 오차를 보상하여 전동기의 제어성능을 높이고, 저가 구현이 중요시 되는 분야에서는 비용을 고려하여 데드타임의 영향을 보상하지 않는 것이 일반적이다. 데드타임의 영향은 주로 저속 운전 영역에서 크기 때문에, 송풍용 팬과 같이 주로 중속 및 고속 영역에서 운전하는 응용 분야에서는 데드타임을 보상하지 않아도 데드타임으로 인한 성능 저하는 미미한 편이다.</p> <p>본 논문에서 제안하는 방식은 전동기의 전류 검출없이 전압의 위상각 제어만으로 추정된 d축 전류가 0이 되도록 하여 단위 전류 당 최대 토크 제어를 구현하는 것이다. 식 (6)에서는 전압 지령과 실제 전동기에 인가되는 전압이 동일하다는 가정 하에 d축 전류를 추정하였다. 그러나 실제로는 데드타임으로 인해 지령 전압과 실제 전동기에 인가되는 전압에는 오차가 발생하고, 이로 인해 식 (6)에 의한 d축 전류 추정에는 오차가 발생하게 된다. 본 논문에서 제안하는 단위 전류 당 최대 토크 제어기에서 d축 전류 추정에 오차가 발생하게 되면 전동기의 운전 점은 단위 전류 당 최대 토크 운전 점에서 벗어나게 되므로 전류 추정 오차에 대한 보상이 필요하다. 본 논문에서는 전류 추정 식에 데드타임에 의한 전압 오차를 반영함으로써 추정 오차를 보상하였다.</p> <p>3 상 전압형 인버터에서 데드타임으로 인한 각 상의 출력 전압 오차 \\( \\left(V_{a, \\text { dead }}, V_{b, \\text { dead }}, V_{c, \\text { dead }}\\right) \\) 는 다음과 같다.</p> <p>\\( V_{a, d e a d}=V_{d e a d} \\cdot \\operatorname{sgn}\\left(i_{a s}\\right) \\)<caption>(13)</caption></p> <p>\\( V_{b, \\text { dead }}=V_{\\text {dead }} \\cdot \\operatorname{sgn}\\left(i_{b s}\\right) \\)<caption>(14)</caption></p> <p>\\( V_{c, \\text { dead }}=V_{\\text {dead }} \\cdot \\operatorname{sgn}\\left(i_{c s}\\right) \\)<caption>(15)</caption></p> <p>\\( \\begin{aligned} V_{\\text {dead }}=& \\frac{T_{\\text {dead }}+T_{o n}-T_{\\text {off }}}{T_{s}} \\cdot\\left(V_{d c}-V_{\\text {sat }}+V_{d}\\right) \\\\ &+\\frac{V_{\\text {sat }}+V_{d}}{2} \\end{aligned} \\)<caption>(16)</caption></p> <p>\\( \\operatorname{sgn}\\left(i_{a s}\\right)=\\left\\{\\begin{array}{ll}1 & : i_{a s}>0 \\\\ -1 & : i_{a s}<0\\end{array}\\right. \\)<caption>(17)</caption></p> <p>상기 식들에서, \\( i_{a s}, i_{b s}, i_{c s}: 3 \\) 상 전류, \\( T_{s} \\) : 인버터의 스위칭 주기, \\( T_{\\text {dead }} \\) : 데드타임, \\( T_{o n}, T_{o f f} \\) : 스위칭 소자의 온, 오프 타임, \\( V_{d c} \\) : 인버터 직류링크 전압, \\( V_{\\text {sat }} \\) : 스위칭 소자의 온 전압 강하, \\( V_{d} \\) : 환류 다이오드의 온 전압 강하이다.각 상의 출력 전압 오차 \\( \\left(V_{a, \\text { dead }}, V_{b, \\text { dea. }}, V_{c, \\text { dead }}\\right) \\) 를 회전자동기좌표계로 변환하여 표현하면 다음과 같다.</p> <p>\\( V_{\\text {d,dead }}^{r}=\\frac{2}{3}\\left(V_{\\text {a,dead }} \\cos \\theta_{e}+V_{\\text {b.dead }} \\operatorname{:os}\\left(\\theta_{e}-\\frac{2}{3} \\pi\\right)\\right. \\) \\( \\left.+V_{\\text {c,dead }} \\operatorname{Cos}\\left(\\theta_{e}+\\frac{2}{3} \\pi\\right)\\right) \\)<caption>(18)</caption></p> <p>\\( V_{q, \\text { dead }}^{r}=-\\frac{2}{3}\\left(V_{\\text {a,dead }} \\sin \\theta_{e}+V_{b \\text {,dead }} \\sin \\left(\\theta_{e}-\\frac{2}{3} \\pi\\right)\\right. \\)\\( \\left.+V_{\\text {c.dead }} \\sin \\left(\\theta_{e}+\\frac{2}{3} \\pi\\right)\\right) \\)<caption>(19)</caption></p> <p>상기 식들에서 \\( V_{d, d e a d}^{r}, V_{q, d e a d}^{r} \\) 는 각각 d축과 q축에서의 데드타임으로 인한 전압 오차로, 전기각으로 360도 구간에서 \\( V_{d, d e a d}^{r} \\) 와 \\( V_{q, d e a d}^{r} \\) 의 변화를 그리면 그림 3과 같다. \\( V_{d, d e a d}^{r} \\) 와 \\( V_{q, d e a d}^{r} \\) 는 전기각으로 \\( 60^{\\circ} \\) 주기로 변화하고, \\( V_{d, d e a d}^{r} \\) 의 경우에는 \\( 60^{\\circ} \\) 주기 내에서의 평균을 구하면 0 이 되는 것을 그림 3에서 확인할 수 있다.</p> <p>\\( V_{q, \\text { dead }}^{r} \\) 의 경우 전기각으로 \\( 0^{\\circ} \\) 에서 \\( 60^{\\circ} \\) 범위 내에서 평균을 구하면 다음과 같다</p> <p>\\( V_{q, \\text { dead,avg }}^{r}=-\\frac{3}{\\pi} \\int_{0}^{\\frac{\\pi}{3}} V_{q, \\text { dead }}^{r} d t=-1.27 V_{\\text {dead }} \\)<caption>(20)</caption></p> <p>데드타임에 의한 d축 전압 오차는 평균이 0 인 반면, q축 전압 오차는 식 (20)에 나타낸 바와 평균이 0이 아니다. 따라서 데드타임에 의한 전류 추정 오차를 보상하기 위해서는 다음과 같이 전류 추정 식에서 데드타임에 의한 q축 전압 오차를 반영해 주어야한다.</p> <p>\\( \\widehat{i_{d}}=\\frac{R_{s} v_{d}^{*}+w_{e} L_{s}\\left(v_{q}^{*}-1.27 V_{\\text {dead }}^{j}\\right)-\\hat{w}_{e}^{2} L_{s} \\lambda_{P M}}{R_{s}^{2}+\\widehat{w_{e}^{2} L_{s}^{2}}} \\)<caption>(21)</caption></p> <p>\\( \\widehat{i_{q}}=\\frac{R_{s}\\left(v_{q}^{*}-1.27 V_{\\text {dead }}\\right)-\\widehat{w_{e}} L_{s} v_{d}^{*}-R_{s} \\widehat{w_{e}} \\lambda_{P M}}{R_{s}^{2}+\\widehat{w_{e}^{2}} L_{s}^{2}} \\)<caption>(22)</caption></p>\n\n<h1>2. 단위 전류 당 최대 토크 운전을 위한 전압 지령의 위상각 제어</h1> <p>공극에서의 자속 분포가 정현적이고, 자계 회로가 선형적이며, 전동기의 제정수가 일정하다고 가정할 때 회전자 동기 좌표계에서의 표면부착형 영구자석 동기전동기의 전압 방정식 및 토크 방정식은 다음과 같다.</p> <p>\\( \\left[\\begin{array}{c}v_{d} \\\\ v_{q}\\end{array}\\right]=\\left[\\begin{array}{cc}R_{s}+p L_{s} & -w_{e} L_{3} \\\\ w_{e} L_{s} & R_{s}+p L_{s}\\end{array}\\right]\\left[\\begin{array}{c}i_{d} \\\\i_{q}\\end{array}\\right]+\\left[\\begin{array}{c}0 \\\\ w_{e} \\lambda_{P M}\\end{array}\\right] \\)<caption>(1)</caption></p> <p>\\( T_{e}=\\frac{3}{2} P \\lambda_{P M} i_{q} \\)<caption>(2)</caption></p> <p>상기 식들에서,\\( \\begin{array}{ll}v_{d}, v_{q} & : \\mathrm{d} \\text { 축 및 } \\mathrm{q} \\text { 축 전압, } \\\\ i_{d}, i_{q} & : \\mathrm{d} \\text { 축 및 } \\mathrm{q} \\text { 축 전류, } \\\\ R_{s} & : \\text { 전기자 권선 저항, } \\\\ L_{s} & : \\text { 인덕턴스, } \\\\ \\omega_{e} & : \\text { 전기 각속도, } \\\\ p & : \\text { 미분 연산자, } \\\\ P & : \\text { Pole pair의 수, } \\\\ T_{e} & : \\text { 전동기 출력 토크, } \\\\ \\lambda_{P M} & : \\text { 역기전력 상수이다. }\\end{array} \\)</p> <p>정상 상태에서 \\( \\mathrm{d} \\) 축 및 \\( \\mathrm{q} \\) 축 전류가 일정하다고 가정하면 \\( p L_{s} i_{d}=0, p L_{s} i_{q}=0 \\) 이 되고, 식 (1)은 다음과 같이 표현할 수 있다.</p> <p>\\( v_{d}=R_{s} i_{d}-w_{e} L_{s} i_{q} \\)<caption>(3)</caption></p> <p>\\( v_{q}=R_{s} i_{q}+w_{e} L_{s} i_{d}+w_{e} \\lambda_{P M} \\)<caption>(4)</caption></p> <p>식 (3)과 식 (4)를 \\( d \\) 축 전류에 관하여 정리하면 다음과 같다.</p> <p>\\( i_{d}=\\frac{R_{s} v_{d}+w_{e} L_{s} v_{u}-w_{s}^{2} L_{s} \\lambda_{P M}}{R_{s}^{2}+w_{e}^{2} L_{s}^{2}} \\)<caption>(5)</caption></p> <p>전동기에 인가되는 d-q축 전압 \\( \\left(v_{d}, v_{q}\\right) \\) 이 전압 지령 \\( \\left(v_{d}^{*}, v_{q}^{*}\\right) \\) 과 동일하고, 전동기의 제정수가 일정하며, 전기각 속도 \\( \\left(\\omega_{e}\\right) \\) 를 검출할 수 있다면 전류 검출 센서 없이도 다음과 같이 표면부착형 영구자석 동기전동기의 d축 전류를 추정할 수 있다.</p> <p>\\( \\hat{i_{d}}=\\frac{R_{s} v_{d}^{*}+w_{e} L_{s} v_{q}^{*}-w_{e}^{2} L_{s} \\lambda_{P M}}{R_{s}^{2}+w_{e}^{2} L_{s}^{2}} \\)<caption>(6)</caption></p> <p>식 (6)에서 \\( \\widehat{i_{d}} \\) 은 추정된 d축 전류이다.</p> <p>마찬가지로 \\( \\mathrm{q} \\) 축 전류도 다음과 같이 추정할 수 있다.</p> <p>\\( \\hat{i_{q}}=\\frac{R_{s} v_{q}^{*}-w_{e} L_{s} v_{d}^{*}-R_{s} w_{e} \\lambda_{P M}}{R_{s}^{2}+w_{e}^{2} L_{s}^{2}} \\)<caption>(7)</caption></p> <p>표면부착형 영구자석 동기전동기의 단위 전류 당 최대 토크 운전은 d축 전류를 0으로 제어함으로써 이루어진다. 본 논문에서 제안하는 표면부착형 영구자석 동기전동기의 단위 전류 당 최대 토크(MTPA) 운전방법은 식 (6)에 의해 d축 전류를 추정하고, 추정된 d축 전류가 0이 되도록 전압 지령의 위상각을 제어하는 것으로, 그 구성은 그림 1과 같다.</p> <p>그림 1 에서 전동기 회전자의 전기각 \\( \\left(\\hat{\\theta_{e}}\\right) \\) 및 각속도 \\( \\left(\\widehat{\\omega_{e}}\\right) \\) 는 홀센서의 출력 신호로부터 추정된다. 3 개의 홀센서를 사용한다고 가정할 경우 전기각으로 \\( 60^{\\circ} \\) 단위의 위치 분해능을 가지므로, 홀센서 출력 신호들이 변화되는 시점 간의 시간 차 \\( \\left(\\delta T_{60}\\right) \\) 를 측정하여 다음과 같이 회전자의 전기적 각속도를 추정할 수 있다.</p> <p>\\( \\hat{\\omega}_{e}=\\frac{\\pi}{3 \\delta T_{60}} \\)<caption>(8)</caption></p> <p>회전자의 기계적인 각속도 \\( \\left(\\omega_{m}\\right) \\) 는 전기적인 각속도를 P(Pole pair)로 나누어서 구할 수 있으며, 회전자의 전기각 \\( \\left(\\hat{\\theta_{e}}\\right) \\) 은 식 (8)에 의해 얻어진 전기적 각속도를 이용하여 다음과 같이 구한다.</p> <p>\\( \\hat{\\theta_{e}}=\\theta_{h}+\\hat{\\omega}_{e} \\Delta T \\)<caption>(9)</caption></p> <p>식 (9)에서 \\( \\theta_{h} \\) 는 홀센서 출력신호가 변화된 시점에서의 회전자의 전기각으로 \\( 0^{\\circ}, 60^{\\circ}, 120^{\\circ}, 180^{\\circ}, 240^{\\circ} \\), \\( 300^{\\circ} \\) 중 하나에 해당하는 값을 가지며, 홀센서 출력신호가 변화할 때마다 새로운 값으로 변경된다. \\( \\Delta T \\) 는 홀센서 출력신호가 변화된 시점을 기준으로 현 시점까지의 경과시간을 의미한다.</p> <p>그림 1 에서 속도 제어기는 지령 속도 \\( \\left(\\omega_{m}^{*}\\right) \\) 와 현재의 기계적인 각속도로부터 오차를 구하고, 그 오차를 0으로 하기 위한 전압 지령 \\( \\left(v_{s}^{*}\\right) \\) 을 출력한다. 전압 위상각 제어기는 다음과 같이 속도 제어기의 출력으로부터 d축 및 q축의 전압 지령 \\( \\left(v_{d}^{*}, v_{q}^{*}\\right) \\) 을 구한다.</p> <p>\\( v_{d}^{*}=-v_{s}^{*} \\sin \\left(\\theta_{\\text {angle }}\\right) \\)<caption>(10)</caption></p> <p>\\( v_{q}^{*}=v_{s}^{*} \\cdot \\cos \\left(\\theta_{\\text {angle }}\\right) \\)<caption>(11)</caption></p> <p>식 (10), (11)에서 \\( \\theta_{\\text {angle }} \\) 는 전압 지령 \\( \\left(v_{s}^{*}\\right) \\) 의 위상각으로 그림 2에 나타낸 바와 같다.</p> <p>표면부착형 영구자석 동기전동기의 단위 전류 당 최대 토크 운전을 위한 d축 전류의 지령 \\( \\left(i_{d}^{*}\\right) \\) 은 0 이다. 따라서 단위 전류 당 최대 토크 운전을 위한 전압 지령의 위상각 \\( \\theta_{\\text {angle }} \\) 는 다음과 같이 적분 제어를 통해서 구할 수 있다.</p> <p>\\( \\theta_{\\text {angle }}=-K_{i} \\int\\left(i_{d}^{*}-\\hat{i_{d}}\\right) d t=K_{i} \\int \\hat{i_{d}} d t \\)<caption>(12)</caption></p> <p>식 (12)에서 \\( K_{i} \\) 는 적분 제어기의 이득이다. 식 (12)에 의해 구해진 전압 지령의 위상각 \\( \\theta_{\\text {angle }} \\) 와 식 (10), (11)에 의한 전압 위상각 제어기의 동작에 대해 예를들어 설명하면, 만약 추정 전류 \\( \\widehat{i_{d}} \\) 이 0 보다 큰 값을 가지면 전류의 위상이 단위 전류 당 최대 토크 운전을 위한 전류 위상보다 뒤지고 있는 것이므로 전압 지령의 위상각 \\( \\theta_{\\text {angle }} \\) 를 증가시켜 추정 전류 \\( \\widehat{i_{d}} \\) 이 0 이 되도록 한다. 반대로, 추정 전류 \\( i_{d} \\) 이 0 보다 작은 값을 가지면 전류의 위상이 앞서고 있는 것이므로 전압 지령의 위상각 \\( \\theta_{\\text {angle }} \\) 를 감소시켜 추정 전류 \\( \\widehat{i_{d}} \\) 이 0 이 되도록 한다. 따라서 본 논문에서 제안한 방식에 의해 전류 검출을 통한 전류 제어 기킁 없이 단순히 전압의 위상각 제어만으로 표면부착형 영구자석 동기전동기의 단위 전류 당 최대 토크 운전을 구현할 수 있다.</p>",
    "여름과 사랑과 청춘에 대한 시를 써줘"
]

In [41]:
completion = client.chat.completions.create(
    model="amphora/fc-proj1-test01",
    messages=[
        {"role": "user", "content": prompts[0]}
    ]
)

print(completion.choices[0].message.content)

물론이죠! '돛단배'로 삼행시 부탁드려요.

돛: 돛을 높이 달고  
단: 단단한 의지로 먼 바다를 향해  
배: 배는 조용히, 그러나 힘차게 나아갑니다.


In [42]:
completion = client.chat.completions.create(
    model="amphora/fc-proj1-test01",
    messages=[
        {"role": "user", "content": prompts[1]}
    ]
)

print(completion.choices[0].message.content)

**배경**  
무선통신 기술의 발전과 모바일 단말기의 소형화 요구에 대응해, 내장형 안테나의 개발이 필수적입니다. 내장형 안테나는 외장형 대비 손상 및 충격에 강하고, 설계상 소형화와 함께 SAR(Specific Absorption Rate) 저감이라는 장점이 있습니다. 다만, 이동통신 단말기에 통합 가능한 소형 안테나는 넓은 대역폭을 확보하는 데 한계가 있습니다. 기존 세라믹 칩, 평면 역 F형, 평면 모노폴 안테나들은 소형화, 경량화 혹은 생산 용이성을 갖추지만, 광대역성과 크기/대역폭의 상충 문제를 내포하고 있습니다.

**문헌적 공백**  
소형화와 광대역 특성의 동시 달성을 위한 기존 연구들은 극한 소재 사용(초고유전율 기판) 혹은 복잡한 구조 설계, 이중/다중 대역 공진을 위한 다양한 변형에 집중돼 왔습니다. 그러나, 단락핀(plane pin) 구조와 슬릿(slit) 도입이 결합된 구조의 체계적인 동작 주파수 및 대역폭 영향 분석은 부족했습니다.

**접근 방법**  
본 논문은 동판 두께 0.2mm, FR4 기판 유전율 4.7 및 0.7874mm 두께, 30mm×10mm 크기의 단락핀 기반 복사판에 슬릿을 부가한 새로운 구조를 제안합니다. CST Microwave Studio를 활용해 단락핀 위치, 슬릿 및 간격에 따른 공진 주파수와 임피던스 대역폭을 시뮬레이션하였고, 제작 후 HP-8753ES 벡터 네트워크 분석기를 이용해 반사 손실, 입력 임피던스 및 이득을 정량 평가했습니다.

**핵심 시사점**  
단락핀의 위치 조절로 공진 주파수를 세밀하게 조정하며, 슬릿 길이 및 단락핀-슬릿 간격 최적화를 통해 1.934GHz에서 대역폭 803MHz(40.8%)을 달성, PCS/IMT-2000/Wibro 대역을 만족하였습니다. 최대 이득 2.62dBi(IMT-2000) 및 1.55dBi(Wibro), 1.52dBi(PCS) 등 우수한 수치를 기록하며, 소형화, 광대역, 내장형 통합에 탁월함을 입증하였습니다. 이 구조는 기존 평면형 모노폴 및 슬릿-카플

In [43]:
completion = client.chat.completions.create(
    model="amphora/fc-proj1-test01",
    messages=[
        {"role": "user", "content": prompts[2]}
    ]
)

print(completion.choices[0].message.content)

취직 시기는 여러 요인에 따라 달라질 수 있습니다. 자신이 어떤 분야에서 일하고 싶은지, 어떤 기술이나 자격을 갖추고 있는지 등에 따라 다릅니다. 다음은 취직 시기를 조금 더 앞당길 수 있는 몇 가지 방법입니다:

1. **이력서 준비**: 자신의 경력과 기술을 잘 나타낼 수 있는 이력서를 준비하세요.
2. **네트워킹**: 관련 분야의 사람들과의 네트워킹을 통해 더 많은 기회를 얻을 수 있습니다. LinkedIn 같은 플랫폼을 활용해보세요.
3. **기술 향상**: 현재 하는 일과 취업 목표 사이의 간극을 좁히기 위해 새로운 기술이나 자격증을 익히는 것도 도움이 될 수 있습니다.
4. **자신의 커뮤니티 참여**: 관심 있는 분야의 온라인 커뮤니티나 포럼에 참여해 정보를 공유하고 다른 사람들과의 관계를 맺어보세요.

지속적으로 자기 자신을 발전시키고 기회를 찾는 것이 중요합니다. Rx재단 포함 다양한 온라인 플랫폼이 있다 보니, 취업에도 요즘은 빠르게 반응할 수 있습니다.


In [44]:
completion = client.chat.completions.create(
    model="amphora/fc-proj1-test01",
    messages=[
        {"role": "user", "content": prompts[3]}
    ]
)

print(completion.choices[0].message.content)

아래는 문제의 조건을 모두 반영한 실행 가능한 Python 함수입니다.

**문제 핵심 요약**
- 왼손 전용: l명  
- 오른손 전용: r명  
- 양손잡이(a명): 양손 중 한쪽에 배치 가능  
- 팀 전체 인원 수는 반드시 짝수여야 함.

**최대 팀 인원수:**
- 가능한 한 많은 왼/오 선수를 채우기  
- 남은 인원이 모두 양손잡이이면, 남은 양손잡이 중 절반은 왼손, 절반은 오른손으로 배치 가능.  
- 모두 소진 후 남개 수는 무시

### 코드

```python
def solve():
    l, r, a = map(int, input().split())

    # 왼손/오른손에 최대한 배치
    left = min(l, r)
    right = min(l - left, r - left)

    # 남은 선수는 모두 양손잡이: 최대만큼 짝수로 더할 수 있음
    total = (left + right) * 2
    print(total)

solve()
```

### 설명

1. **왼손/오른손에 각각 배치 가능한 최대 인원수:**  
    - 배정 가능한 인원: `min(l, r)`  
    - 남은 인원: `l - min(l, r)` 또는 `r - min(l, r)`  
    - 이들 모두를 양손 중 하나로 배치할 수 있음  
2. **최대 팀 인원:**  
   - 가능한 만큼 한쪽에 배치, 나머지는 양손잡이를 남은 자리만큼 추가 짝수로 채움

---

예제 입력/출력을 함수가 모두 올바르게 처리합니다.


In [45]:
completion = client.chat.completions.create(
    model="amphora/fc-proj1-test01",
    messages=[
        {"role": "user", "content": prompts[4]}
    ]
)

print(completion.choices[0].message.content)

**배경**  
영구자석 동기전동기(PMSM)는 구조가 단순하고 고효율을 달성하여 산업용 팬 등 다양한 응용처에서 널리 쓰이나, 단위 전류 당 최대 토크(MTPA) 운전을 위해서는 d축Direct-axis 전류 제어가 필수적이다. 이 목적을 위해서는 디지털 제어와 전류 센서, 고성능 마이크로프로세서가 반드시 필요하여 시스템 비용 상승이 문제로 대두된다. 전류 제어 없이 단순 전압 제어 방식이 적용되어 왔으나, 룩업테이블 기반 위상각 조정이 복잡성 및 실시간성에서 한계가 있었다.

**문헌적 공백**  
기존의 션트 저항 또는 홀센서-기반 피크 검출법은 전류 검출 대안으로 제시되었으나, 션트 저항 방식은 노이즈 및 필터링에 취약하고, 홀센서-기반 방식 역시 노이즈/오프셋에 민감하다. 또한, 수학적 모델 기반 전류 추정(특히 데드타임 보상 포함)에서 추정 오차를 보상하는 체계적 방안이 미흡하다.

**접근 방법**  
본 논문은 정현적 자속분포 및 선형 자계회로 가정 하 내부 디지털 제어나 센서 없이, 인가 전압의 위상각 제어만으로 d축 전류 제어를 우회하는 적분 제어 기법을 제안한다. 추정 d축 전류가 d축 전류 지령(0)과 일치하도록 전압 위상각(\(\theta_{angle}\))을 적분적으로 보정한다. 데드타임 동안 인버터 각상 출력 전압의 변화가 d축과 q축(상대좌표)에서 각각 다르게 나타나는 점을 수식적으로 도출하고, 이 평균 0이 아닌 q축 오차(\(-1.27V_{dead}\))만을 전류 추정식에 보정항으로 적용한다. 전기각 및 속도는 홀센서 신호의 시간적 변화로부터 직접적으로 추정한다.

**핵심 시사점**  
제안기법은 센서리스 MTPA 제어로서 디지털 제어기, 고속연산, 센서 없이도 산업용 표면부착형 영구자석 동기전동기 운전의 핵심 성능(토크추종, 효율)을 확보할 수 있음을 보인다. 데드타임 오차의 질적 분석 및 보정수식을 최초로 제시함으로써 모델-기반 저가형 PMSM제어링场에 실질적 적용 가능성을 제공한다.


In [46]:
completion = client.chat.completions.create(
    model="amphora/fc-proj1-test01",
    messages=[
        {"role": "user", "content": prompts[5]}
    ]
)

print(completion.choices[0].message.content)

여름과 사랑과 청춘에 대한 시

뜨거운 여름, 하늘은 맑고  
새들은 높이 날며 노래해.  
아무도 머뭇거리지 않는 이 계절,  
아무에게도 뒤처지지 않는 우리들.

순수한 마음을 품은 청춘의 계절,  
너와 나, 서로에게 반해  
여름밤 별빛 아래 작은 소망을 품어.  
햇살 아래 서로의 등을 밀며  
우리의 사랑은 조금씩 자라나.

언제나 여름이면 찾아오는 설렘,  
천둥 번개 치는 오후,  
너와 손잡고 뛰던 그 거리의 소음도  
사랑이었지, 그때의 우리는.

시간은 흘러도 추억은 남아  
따뜻한 바람결에 실려와,  
사진 속 우리 웃는 모습에  
청춘은 살아 있네, 아직도.

사랑은 여름처럼 짧기도 하고  
청춘은 겨울처럼 길기도 하네.  
여름날의 너와 나,  
이 계절 안에 영원히 빛나리.
